### 方法
```
while 調べるべき特徴量がある限り :
    for 調べる特徴量 in 調べる特徴量の数 :
        ある特徴量ベクトルxiと元のy(y^)の内積を求める(t^i)
        for 100程度 :
            y^を、{t0 = t^0かつt1 = t^1...tj = t^j}という条件の元サンプリング（nealを使う）しyを生成する。
            ある特徴量ベクトルxiとyの内積を求める(ti)
        複数のtiを求めることによって得られた分布に対するt^iの位置から、p値を求める。
    p値の一番小さいXsを知る。
    条件{}にts = t^sを足す（更新する）
    調べる特徴量からXsを除外（更新する）
```

#### 条件について
- t0 = t^0 : 1の総和が同じ
- tj = t^j : xの特徴量jつ目とyの内積の値が同じ

### データ
mushroomの50サンプル分

### yのサンプリング
- コンパイルを行うまでは`pyqubo`
- SAは`SimulatedAnnealingSampler()`

### 改訂(08/03)
- ori_tiのkeyがなかった場合に1とする必要がない
- sum([j[1] for j in ti_list])は必ず100である

### 改訂（08/04）
- samplingでnum=100とする。


### 改訂（2020/08/18）
- mushroom_lの1の総数を8にしよう。(今までは3だった)
- A4を2枚
- 幅を付けた図を描く
- tを0, 1と増やす

### 改訂（2020/08/19）
- 8/18に新たに作ったデータではt^1のとき該当するyが一つであったため非常に難しいと判断。再度作り直して実験することにした。

### 改訂（2020/08/24）
- 交絡因子はデータ分析する前からわかっているそう。（そうだとわかっていなかった）
- t^0のターンではP値を計算しない。t^1のターンでは事前に決まっている交絡因子を使って各変数のP値を計算する。
- 8/18に載せたデータを使って、交絡因子を'4'とする

In [34]:
#!pip3 install dwave-neal

In [35]:
import pandas as pd 
import numpy as np
from neal import SimulatedAnnealingSampler
from pyqubo import Array, Constraint, Placeholder, solve_qubo
import sys
import pprint

In [36]:
X_mushroom = pd.read_csv("../input/X_l_mushroom.csv", sep=',', index_col=0) #50bit
y_mushroom = pd.read_csv('../input/y_l_mushroom.csv', sep=',', index_col=0)['21']

num_samples = X_mushroom.shape[0]
num_cols = X_mushroom.shape[1]
columns = list(X_mushroom.columns)

y_sum = sum(y_mushroom)

#交絡因子として設定
j = '4'

-----------

## t^0条件だけでのサンプリング
- sumの条件だけを入れる。
- 最小のP値をもつ変数を知る必要はない。
- t^0を回したところで'4'だとわかっている。

## t^1も追加したサンプリング、P値計算
- t^0条件とt^1条件を入れてサンプリング、p値計算
- t^1条件とは'4'を使った条件

In [37]:
#numreadsは作るサンプル数（ダブリはあるはず）
#numreadsは100, 500, 1000とするか、logをとるか(np.logspace)

numreads_10 = np.logspace(1, 4, num=4, base=10, dtype=int)
len(numreads_10) #4
print(numreads_10)

numreads_5 = [ll*100 for ll in range(1, 21) if ll%5==0 or ll==1]
len(numreads_5) #5
print(numreads_5)

[   10   100  1000 10000]
[100, 500, 1000, 1500, 2000]


In [93]:
minP_keys_dic = {}

In [100]:
servey_num_col_list = [jj for jj in columns if jj != j]
qbl_p_list = [] ##
for k in range(10): #10回回して平均、分散を得る
    y = Array.create('y', shape=num_samples, vartype='BINARY')

    #QUBO式で定式化
    H = (sum(y) - y_sum)**2
    H_plus = (np.dot(X_mushroom[str(j)], y) - np.dot(X_mushroom[str(j)], y_mushroom))**2
    H = H + H_plus    
    qb = H.compile().to_qubo()
    sa_sampler = SimulatedAnnealingSampler()
    
    #res = sa_sampler.sample_qubo(qb[0], num_reads=numreads_10[3]) #0, 1, 2, 3
    res = sa_sampler.sample_qubo(qb[0], num_reads=numreads_5[4]) #1, 3, 4
    
    #fig, ax = plt.subplots(int(num_cols/2), 2, figsize=(30,30))

    qb_minP_cols = []
    qbl_p = {} ##
    
    for i in servey_num_col_list: #1, 2, ..., 
        ori_ti = int(np.dot(X_mushroom[i], y_mushroom)) #特徴量iを表すp

        ti_dic = {} #{内積の値:該当数}という辞書。

        valid_y_num = 0
        for y_info in list(res.record):
            if sum(y_info[0]) - y_sum == 0: #サンプリングした結果が本当に条件に適しているのか確認
                valid_y_num += 1
                y = pd.Series(y_info[0])

                if int(np.dot(X_mushroom[i], y)) in ti_dic.keys():
                    ti_dic[int(np.dot(X_mushroom[i], y))] += 1
                else:
                    ti_dic[int(np.dot(X_mushroom[i], y))] = 1

        ti_list = sorted(ti_dic.items()) #listになり、要素はタプルになる。
        franc = sum([j[1] for j in ti_list if j[0] >= ori_ti])
        #franc = sum([j[1] for j in ti_list if j[0] > ori_ti]) #変更してみた2020/08/21
        qbl_p[i] = franc/valid_y_num
        
    ##一周終了
    qbl_p_list.append(qbl_p) ##

## 10回終了
to_qubol_p_10times = pd.DataFrame(qbl_p_list)
print(to_qubol_p_10times)

#最小値のpの列名取得（typeはarray)
ll = np.where(to_qubol_p_10times.mean() == to_qubol_p_10times.mean().min())[0]
minP_keys = [servey_num_col_list[l] for l in ll]

#minP_keys_dic[numreads_10[3]]=minP_keys #0, 1, 2, 3
minP_keys_dic[numreads_5[4]]=minP_keys #1, 3, 4
print(minP_keys_dic)

#to_qubol_p_10times.to_csv('../output/mushrooml/to_qubol_10samples_p_10time_conf4.csv')
#to_qubol_p_10times.to_csv('../output/mushrooml/to_qubol_100samples_p_10time_conf4.csv')
#to_qubol_p_10times.to_csv('../output/mushrooml/to_qubol_1000samples_p_10time_conf4.csv')
#to_qubol_p_10times.to_csv('../output/mushrooml/to_qubol_10000samples_p_10time_conf4.csv')


##to_qubol_p_10times.to_csv('../output/mushrooml/to_qubol_100samples_p_10time_conf4.csv')
#to_qubol_p_10times.to_csv('../output/mushrooml/to_qubol_500samples_p_10time_conf4.csv')
##to_qubol_p_10times.to_csv('../output/mushrooml/to_qubol_1000samples_p_10time_conf4.csv')
#to_qubol_p_10times.to_csv('../output/mushrooml/to_qubol_1500samples_p_10time_conf4.csv')
to_qubol_p_10times.to_csv('../output/mushrooml/to_qubol_2000samples_p_10time_conf4.csv')

     0         1         2    3         5         6    7         8         9  \
0  1.0  0.001001  0.783892  1.0  0.613307  0.333167  1.0  0.891446  0.792896   
1  1.0  0.005003  0.794397  1.0  0.610805  0.369185  1.0  0.899450  0.791396   
2  1.0  0.002000  0.790000  1.0  0.601000  0.325000  1.0  0.905000  0.775500   
3  1.0  0.002503  0.771271  1.0  0.613614  0.338338  1.0  0.900400  0.790791   
4  1.0  0.002500  0.796000  1.0  0.601000  0.326000  1.0  0.894500  0.778000   
5  1.0  0.001500  0.787000  1.0  0.609000  0.340000  1.0  0.905000  0.778500   
6  1.0  0.002000  0.790500  1.0  0.617500  0.345000  1.0  0.895500  0.787000   
7  1.0  0.001502  0.782783  1.0  0.626126  0.332332  1.0  0.908909  0.757758   
8  1.0  0.001000  0.788500  1.0  0.591500  0.343000  1.0  0.893000  0.793500   
9  1.0  0.002500  0.795500  1.0  0.619500  0.339000  1.0  0.908500  0.792000   

    10  ...  109       110  111       112       113  114  115  116  117  \
0  1.0  ...  1.0  0.900950  1.0  0.787394  0

In [86]:
myDF = pd.DataFrame({
    "1": [1, 1, 1, 0, 0],
    "2": [1, 0, 1, 0, 1],
    "3": [0, 0, 0, 0, 0], #
    "5": [0, 1, 0, 0, 0],
    "6": [0, 0, 0, 0, 0]#
})
myDF

,1,2,3,5,6
0,1,1,0,0,0
1,1,0,0,1,0
2,1,1,0,0,0
3,0,0,0,0,0
4,0,1,0,0,0


In [88]:
ll = np.where(myDF.mean() == myDF.mean().min())[0]
ll

array([2, 4])

In [89]:
myDF_columns = ['1', '2', '3', '5', '6']

In [90]:
minP_keys = [myDF_columns[l] for l in ll]
minP_keys

['3', '6']

In [55]:
np.where(to_qubol_p_10times.mean() == to_qubol_p_10times.mean().min())

(array([ 1, 26]),)

In [50]:
to_qubol_p_10times['1']

0    0.001701
1    0.001700
2    0.002000
3    0.002000
4    0.001300
5    0.001801
6    0.002000
7    0.002101
8    0.001501
9    0.001601
Name: 1, dtype: float64

In [51]:
to_qubol_p_10times['27']

0    0.001701
1    0.001700
2    0.002000
3    0.002000
4    0.001300
5    0.001801
6    0.002000
7    0.002101
8    0.001501
9    0.001601
Name: 27, dtype: float64

In [52]:
to_qubol_p_10times['26']

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
5    1.0
6    1.0
7    1.0
8    1.0
9    1.0
Name: 26, dtype: float64

-----

##  t^1を行う
- 交絡因子jとして4を採用。（糖尿病でいう、性別のような立ち位置）
- よって、t^0の条件（sumが同じ）かつt^1の条件（X[4]と元yの内積==X[4]とサンプリングされたyの内積）となるようなyを採用する。
- サンプリングされた後、P値を計算しyと相関の大きい変数を見つける

- 平均などt^0のときと同じ図を出力する
- 一番小さいiを知る

In [9]:
def make_y(j, numreads):
    
    # BINARY変数
    y = Array.create('y', shape=num_samples, vartype='BINARY')
    
    # t0の条件
    H = (sum(y) - y_sum)**2
    
    # tのそれ以外の条件
    for i in list(j):
        H_plus = (np.dot(X_mushroom[str(i)], y) - np.dot(X_mushroom[str(i)], y_mushroom))**2
        H = H + H_plus
    
    qb = H.compile().to_qubo()
    sa_sampler = SimulatedAnnealingSampler()

    res = sa_sampler.sample_qubo(qb[0], num_reads=numreads)
    
    valid_y_list = []
    for y_info in list(res.record):
        if sum(y_info[0]) - y_sum ==0:
            if np.dot(X_mushroom[str(j)], pd.Series(y_info[0])) - np.dot(X_mushroom[str(j)], y_mushroom)==0:
                valid_y_list.append(pd.Series(y_info[0]))
                    
    return valid_y_list

In [10]:
#numreadsの違いによって、tをいくつまで(0-)回せるのか、minPlistはどう変わるのか、確認する
t_num_dic = {}

In [12]:
qbl_p_list = [] ##
    for k in range(20): #10回回して平均、分散を得る
        valid_y_list = make_y(j, numreads_10[3]) #0, 1, 2, 3
        #valid_y_list = make_y(j, numreads_5[4]) #1, 3, 4
        
        if len(valid_y_list) == 0:
            continue
        
        qbl_p = {} ##
        for i in servey_num_col_list:
            ori_ti = int(np.dot(X_mushroom[i], y_mushroom)) #特徴量iを表すp

            ti_dic = {} #{内積の値:該当数}という辞書。
            
            for y in valid_y_list:      
                if int(np.dot(X_mushroom[i], y)) in ti_dic.keys():
                    ti_dic[int(np.dot(X_mushroom[i], y))] += 1
                else:
                    ti_dic[int(np.dot(X_mushroom[i], y))] = 1

            ti_list = sorted(ti_dic.items()) #listになり、要素はタプルになる。
            franc = sum([j[1] for j in ti_list if j[0] >= ori_ti])
            #franc = sum([j[1] for j in ti_list if j[0] >= ori_ti]) #変更してみた2020/08/21
            qbl_p[i] = franc/len(valid_y_list)
        

        ##一周終了
        qbl_p_list.append(qbl_p) ##
        
        if len(qbl_p_list) == 10:
            break
            
    ## 10回終了
    to_qubol_p_10times = pd.DataFrame(qbl_p_list)
    
    if to_qubol_p_10times.shape[0] < 10:
        t_num_dic[numreads_10[3]] = [tt-1, minP_cols] #0, 1, 2, 3
        #t_num_dic[numreads_5[4]] = [tt-1, minP_cols] #1, 3, 4
        print(t_num_dic)
        print('----------------')
        break
    
    else:
        print(to_qubol_p_10times) #表示
    
    #最小値のpの列名取得（typeはarray)
    minP_keys = np.where(to_qubol_p_10times.mean() == to_qubol_p_10times.mean().min())[0]
    print(list(minP_keys)) #表示
    
    minP_cols += list(minP_keys)

    servey_num_col_list = [i for i in servey_num_col_list if i not in minP_keys]
    
    tt +=1
    print(tt) #表示
    
#to_qubol_p_10times.to_csv('../output/mushrooml/t0t1_qubol_10samples_p_10time_conf4.csv')
#to_qubol_p_10times.to_csv('../output/mushrooml/t0t1_qubol_100samples_p_10time_conf4.csv')
#to_qubol_p_10times.to_csv('../output/mushrooml/t0t1_qubol_1000samples_p_10time_conf4.csv')
to_qubol_p_10times.to_csv('../output/mushrooml/t0t1_qubol_10000samples_p_10time_conf4.csv')

##to_qubol_p_10times.to_csv('../output/mushrooml/t0t1_qubol_100samples_p_10time_conf4.csv')
#to_qubol_p_10times.to_csv('../output/mushrooml/t0t1_qubol_500samples_p_10time_conf4.csv')
##to_qubol_p_10times.to_csv('../output/mushrooml/t0t1_qubol_1000samples_p_10time_conf4.csv')
#to_qubol_p_10times.to_csv('../output/mushrooml/t0t1_qubol_1500samples_p_10time_conf4.csv')
#to_qubol_p_10times.to_csv('../output/mushrooml/t0t1_qubol_2000samples_p_10time_conf4.csv')

        0    1         2         3    4         5         6    7         8  \
0  0.9999  0.0  0.693800  0.764500  0.0  0.005500  0.140600  0.0  0.492000   
1  0.9999  0.0  0.698170  0.771577  0.0  0.004800  0.138014  0.0  0.504450   
2  1.0000  0.0  0.689238  0.765453  0.0  0.004801  0.137327  0.0  0.501400   
3  0.9998  0.0  0.692508  0.759328  0.0  0.005602  0.139642  0.0  0.491047   
4  0.9998  0.0  0.691238  0.756051  0.0  0.006301  0.138628  0.0  0.492699   
5  0.9998  0.0  0.692369  0.766877  0.0  0.004500  0.143614  0.0  0.502750   
6  0.9999  0.0  0.693477  0.766106  0.0  0.005002  0.142257  0.0  0.502801   
7  0.9998  0.0  0.697609  0.756727  0.0  0.005702  0.138342  0.0  0.503351   
8  1.0000  0.0  0.696539  0.771154  0.0  0.005201  0.146029  0.0  0.502601   
9  1.0000  0.0  0.694939  0.767654  0.0  0.005001  0.136227  0.0  0.506601   

          9  ...  107  108       109       110  111       112       113  \
0  0.308900  ...  0.0  0.0  0.766500  0.282300  0.0  0.006700  0.8

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:58: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


{10000: [0, [1, 4, 7, 10, 11, 15, 16, 17, 19, 22, 23, 25, 26, 27, 30, 31, 32, 34, 35, 36, 37, 38, 41, 43, 44, 45, 46, 48, 49, 51, 56, 57, 60, 63, 64, 65, 66, 67, 70, 72, 73, 74, 76, 77, 79, 80, 81, 82, 83, 85, 86, 87, 88, 89, 91, 93, 94, 95, 96, 97, 100, 101, 102, 103, 105, 106, 107, 108, 111, 115]]}
----------------


#### 考察
numreads = 10のとき
- サンプル数が足りず、27という特徴量もp値が小さくなった。t^1の際は1, 27両方の特徴量についての条件を満たすyは一つ存在するが得られなかったよう。

numreads = 100のとき
- サンプル数が足りず、27という特徴量もp値が小さくなった。t^1の際は1, 27両方の特徴量についての条件を満たすyは一つ存在するが得られなかったよう。

numreads = 1000のとき
- 1だけ選ばれた。実際は1の特徴量についての条件を満たすyは一つ存在するのだが得られなかったよう。

numreads = 500のとき
- 1だけ選ばれた。実際は1の特徴量についての条件を満たすyは一つ存在するのだが得られなかったよう。

numreads = 1500のとき
- 1だけ選ばれた。実際は1の特徴量についての条件を満たすyは一つ存在するのだが得られなかったよう。

numreads = 2000のとき
- 1だけ選ばれた。実際は1の特徴量についての条件を満たすyは一つ存在するのだが得られなかったよう。

numreads = 10000のとき
- 1だけ選ばれた。実際は1の特徴量についての条件を満たすyは一つ存在するのだが得られなかったよう。

In [162]:
t_num_10times_t0t1 = pd.DataFrame(t_num_dic, index = ['last t num', 'minP'])
t_num_10times_t0t1

,10,100,1000,500,1500,2000,10000
last t num,0,0,0,0,0,0,0
minP,"[1, 27]","[1, 27]",[1],[1],[1],[1],[1]


In [ ]:
#t_num_10times_t0t1.to_csv('../output/mushrooml/t_num_10times_t0t1.csv')

## t（条件）をすべて使ってみる

In [ ]:
minP_cols = []
servey_num_col_list = columns

while len(servey_num_col_list) > 0:    
    qbl_p_list = [] ##
    for k in range(20): #10回回して平均、分散を得る
        #valid_y_list = make_y(minP_cols, numreads_10[2]) #0, 1, 2, 3
        valid_y_list = make_y(minP_cols, numreads_5[3]) #1, 3, 4
        
        if len(valid_y_list) == 0:
            continue
        
        qb_minP_cols = []
        qbl_p = {} ##

        for i in servey_num_col_list:
            ori_ti = int(np.dot(X_mushroom[i], y_mushroom)) #特徴量iを表すp

            ti_dic = {} #{内積の値:該当数}という辞書。
            
            for y in valid_y_list:      
                if int(np.dot(X_mushroom[i], y)) in ti_dic.keys():
                    ti_dic[int(np.dot(X_mushroom[i], y))] += 1
                else:
                    ti_dic[int(np.dot(X_mushroom[i], y))] = 1

                ti_list = sorted(ti_dic.items()) #listになり、要素はタプルになる。
                franc = sum([j[1] for j in ti_list if j[0] >= ori_ti])
                qbl_p[i] = franc/len(valid_y_list)
        

        ##一周終了
        qbl_p_list.append(qbl_p) ##
        
        if len(qbl_p_list) == 10:
            break
            
    ## 10回終了
    to_qubol_p_10times = pd.DataFrame(qbl_p_list)
    
    if to_qubol_p_10times.shape[0] < 10:
        #t_num_dic[numreads_10[2]] = [tt-1, minP_cols] #0, 1, 2, 3
        t_num_dic[numreads_5[3]] = [tt-1, minP_cols] #1, 3, 4
        print(t_num_dic)
        print('----------------')
        break
    
    else:
        print(to_qubol_p_10times) #表示
    
    #最小値のpの列名取得（typeはarray)
    minP_keys = np.where(to_qubol_p_10times.mean() == to_qubol_p_10times.mean().min())[0]
    print(list(minP_keys)) #表示
    
    minP_cols += list(minP_keys)

    servey_num_col_list = [i for i in servey_num_col_list if i not in minP_keys]
    
    tt +=1
    print(tt) #表示
    
#to_qubol_p_10times.to_csv('../output/mushrooml/t0t1_qubol_10samples_p_10time.csv')
#to_qubol_p_10times.to_csv('../output/mushrooml/t0t1_qubol_100samples_p_10time.csv')
#to_qubol_p_10times.to_csv('../output/mushrooml/t0t1_qubol_1000samples_p_10time.csv')
#to_qubol_p_10times.to_csv('../output/mushrooml/t0t1_qubol_10000samples_p_10time.csv')まだ

##to_qubol_p_10times.to_csv('../output/mushrooml/t0t1_qubol_100samples_p_10time.csv')
#to_qubol_p_10times.to_csv('../output/mushrooml/t0t1_qubol_500samples_p_10time.csv')
##to_qubol_p_10times.to_csv('../output/mushrooml/t0t1_qubol_1000samples_p_10time.csv')
to_qubol_p_10times.to_csv('../output/mushrooml/t0t1_qubol_1500samples_p_10time.csv')
#to_qubol_p_10times.to_csv('../output/mushrooml/t0t1_qubol_2000samples_p_10time.csv')

In [7]:
minP_cols = []
servey_num_col_list = columns
distri_element_num = 100

while len(servey_num_col_list) > 0:    
    p = {}
    for i in servey_num_col_list:
        ori_ti = int(np.dot(X_mushroom[i], y_mushroom)) #特徴量iを表すp
        
        ti_dic = {} #{内積の値:該当数}という辞書。
        
        for _ in range(distri_element_num): 
            y = make_y(minP_cols)
            
            if int(np.dot(X_mushroom[i], y)) in ti_dic.keys():
                ti_dic[int(np.dot(X_mushroom[i], y))] += 1
            else:
                ti_dic[int(np.dot(X_mushroom[i], y))] = 1
                
        count_better_t = sum([v for k, v in ti_dic.items() if k >= ori_ti])
        p[i] = count_better_t / distri_element_num

        
    minP_keys = [k for k, v in p.items() if v == min(list(p.values()))]
    minP_cols += minP_keys
    servey_num_col_list = [i for i in servey_num_col_list if i not in minP_keys]